In [1]:
import os

package_path = '/home/meganorm-smkia/Code/MEGaNorm/'
os.chdir(package_path)

from utils.parallel import submit_jobs, check_jobs_status, collect_results
from datasets.camcan import load_camcan_data
from utils.nm import hbr_data_split
from plots.plots import plot_nm_range2

import pcntoolkit as ptk

In [2]:
### Paths and configs

username = 'meganorm-smkia'

mainParallel_path = os.path.join(package_path, 'src', 'mainParallel.py')

data_path = '/project/meganorm/Data/camcan/CamCAN/cc700/meg/pipeline/release005/BIDSsep/derivatives_rest/aa/AA_movecomp_transdef/aamod_meg_maxfilt_00003/'
target_dir = '/home/meganorm-smkia/Data/CamCAN/Features/'

log_path = '/home/meganorm-smkia/temp/log/'
temp_path = '/home/meganorm-smkia/temp/tmp/'

processing_dir = '/home/meganorm-smkia/Data/CamCAN/NM/'

job_configs = {'log_path':log_path, 'module':'mne', 'time':'1:00:00', 'memory':'20GB', 
                'partition':'normal', 'core':1, 'node':1, 'batch_file_name':'batch_job'}

if not os.path.isdir(log_path):
    os.makedirs(log_path)
    
if not os.path.isdir(processing_dir):
    os.makedirs(processing_dir)


In [4]:
### Parallel feature extraction

subjects = os.listdir(data_path)
 
# Running Jobs
start_time = submit_jobs(mainParallel_path, target_dir, data_path, subjects, 
                temp_path, job_configs=job_configs)
# Checking jobs
failed_job_names= check_jobs_status(username, start_time)

while len(failed_job_names)>0:
    # Re-running Jobs
    start_time = submit_jobs(mainParallel_path, target_dir, data_path, failed_job_names, 
                temp_path, job_configs=job_configs)
    # Checking jobs
    failed_job_names= check_jobs_status(username, start_time)

collect_results(target_dir, subjects, temp_path, file_name='camcan_features')

Submitted batch job 12238179
Submitted batch job 12238180
Submitted batch job 12238181
Submitted batch job 12238182
Submitted batch job 12238183
Submitted batch job 12238184
Submitted batch job 12238185
Submitted batch job 12238187
Submitted batch job 12238188
Submitted batch job 12238189
Submitted batch job 12238190
Submitted batch job 12238191
Submitted batch job 12238192
Submitted batch job 12238193
Submitted batch job 12238194
Submitted batch job 12238195
Submitted batch job 12238196
Submitted batch job 12238197
Submitted batch job 12238198
Submitted batch job 12238199
Submitted batch job 12238200
Submitted batch job 12238201
Submitted batch job 12238202
Submitted batch job 12238203
Submitted batch job 12238204
Submitted batch job 12238205
Submitted batch job 12238206
Submitted batch job 12238207
Submitted batch job 12238208
Submitted batch job 12238209
Submitted batch job 12238210
Submitted batch job 12238211
Submitted batch job 12238212
Submitted batch job 12238213
Submitted batc

In [6]:
### Data preparation for Normative Modeling

camcan_cov_path = '/project/meganorm/Data/camcan/CamCAN/cc700/participants.tsv'
camcan_feature_path = '/home/meganorm-smkia/Data/CamCAN/Features/camcan_features.csv'

camcan_data = load_camcan_data(camcan_feature_path, camcan_cov_path)

biomarker_num = hbr_data_split(camcan_data, processing_dir, drop_nans=True, batch_effects=['gender','site'])


In [ ]:
### Setting up NM configs

python_path = '/project/meganorm/Software/Miniconda3/envs/mne/bin/python' 

hbr_configs = {'hetero_SHASH_bspline':{'model_type':'bspline', 'likelihood':'SHASHb', 'linear_sigma':'True',
                                   'random_slope_mu':'True', 'linear_epsilon':'True', 'linear_delta':'True'},
            }

inscaler='minmax' 
outscaler='minmax' 
batch_size = 1
outputsuffix = '_estimate'

respfile = processing_dir + 'y_train.pkl'
covfile = processing_dir + 'x_train.pkl'

testrespfile_path = processing_dir + 'y_test.pkl'
testcovfile_path = processing_dir + 'x_test.pkl'

trbefile = processing_dir + 'b_train.pkl'
tsbefile = processing_dir + 'b_test.pkl'

memory = '2gb'
duration = '2:00:00'
method = 'hetero_SHASH_bspline'
cluster_spec = 'slurm'

In [ ]:
### Running NM

ptk.normative_parallel.execute_nm(processing_dir, python_path,
               'NM', covfile, respfile, batch_size, memory, duration, alg='hbr', 
               log_path=log_path, binary=True, testcovfile_path=testcovfile_path, 
               testrespfile_path=testrespfile_path,trbefile=trbefile, tsbefile=tsbefile, 
               model_type=hbr_configs[method]['model_type'], likelihood=hbr_configs[method]['likelihood'],  
               linear_sigma=hbr_configs[method]['linear_sigma'], random_slope_mu=hbr_configs[method]['random_slope_mu'],
               linear_epsilon=hbr_configs[method]['linear_epsilon'], linear_delta=hbr_configs[method]['linear_delta'], 
               savemodel='True', inscaler=inscaler, outscaler=outscaler, outputsuffix=outputsuffix, 
               interactive='auto', cluster_spec=cluster_spec)

In [ ]:
### Plotting ranges
for i in range(biomarker_num):
    plot_nm_range2(processing_dir, ind=i, parallel=True)